## Quiz 1: Доверительные интервалы

### Question 1

Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины X∼N(μ,σ2) лежит в интервале μ±c⋅σ. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки.

In [1]:
import scipy.stats
import numpy as np

In [2]:
alpha = 1 - 0.997

In [3]:
z = scipy.stats.norm.ppf(1 - alpha/2.0)
z

2.9677379253417717

### Question 5

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [4]:
# плацебо
n1 = 11034.0
a = 189.0

# аспирин
n2 = 11037.0
b = 104.0

In [5]:
p1 = a/n1
p2 = b/n2

In [6]:
p1 - p2

0.0077060239760047815

### Question 6

Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.

In [7]:
# выборка принимавших плацебо
sample1 = np.zeros(int(n1))
sample1[:int(a)] = 1

In [8]:
# выборка принимавших аспирин
sample2 = np.zeros(int(n2))
sample2[:int(b)] = 1

In [9]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [10]:
print "confidence interval: [%f, %f]" % proportions_confint_diff_ind(sample1, sample2)

confidence interval: [0.004688, 0.010724]


### Question 7

Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину p/(1−p), которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить p^. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

(189/11034) / (1 - 189/11034) ≈ 0.0174
Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [11]:
odds1 = p1/(1 - p1)
odds2 = p2/(1 - p2)

In [12]:
odds1/odds2

1.8320539419087138

### Question 8

Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

- составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
- установите random seed=0;
- сделайте по 1000 псевдовыборок из каждой группы пациентов.

In [13]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [14]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [15]:
np.random.seed(0)
bootstrap_samples1 = get_bootstrap_samples(sample1, 1000)
bootstrap_samples2 = get_bootstrap_samples(sample2, 1000)

In [16]:
def odds_func(sample): 
    p = float(sum(sample))/len(sample)
    return p / (1 - p)

In [17]:
odds_sample1 = map(odds_func, bootstrap_samples1)
odds_sample2 = map(odds_func, bootstrap_samples2)

**Интервальная оценка шанса инфаркта**

In [18]:
print "95% confidence interval without aspirin: ", stat_intervals(odds_sample1, 0.05)
print "95% confidence interval with aspirin: ", stat_intervals(odds_sample2, 0.05)

95% confidence interval without aspirin:  [ 0.01480732  0.0198748 ]
95% confidence interval with aspirin:  [ 0.00776114  0.01126993]


**Интервальная оценка отношения шансов инфаркта**

In [19]:
odds_ratio = map(lambda odds: odds[0]/odds[1] , zip(odds_sample1, odds_sample2))

In [20]:
print "95% confidence interval for ration between odds: ", stat_intervals(odds_ratio, 0.05)

95% confidence interval for ration between odds:  [ 1.46286276  2.35093673]
